## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/people.json"
file_type = "json"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

age,name
null,Michael
30,Andy
19,Justin


In [0]:
# Create a view or table

temp_table_name = "people_json"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `people_json`

age,name
null,Michael
30,Andy
19,Justin


In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

data = "people_json"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
df = spark.read.json('/FileStore/tables/people.json')

In [0]:
df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [0]:
df.printSchema()

root
-- age: long (nullable = true)
-- name: string (nullable = true)

In [0]:
df.columns

Out[10]: ['age', 'name']

In [0]:
df.describe().show()

+-------+------------------+-------+
summary| age| name|
+-------+------------------+-------+
 count| 2| 3|
 mean| 24.5| null|
 stddev|7.7781745930520225| null|
 min| 19| Andy|
 max| 30|Michael|
+-------+------------------+-------+

In [0]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [0]:
# Changing the datatype in the dataframe by setting up the schema.
# Creating a Structure Field
data_schema = [StructField('age', IntegerType(), True), 
               StructField('name', StringType(), True)]
# assigning age column to int type instade of logn and Ture is for columns can have null values

In [0]:
final_struct = StructType(fields = data_schema)

In [0]:
df = spark.read.json("/FileStore/tables/people.json", schema = final_struct)

In [0]:
df.printSchema()

root
-- age: integer (nullable = true)
-- name: string (nullable = true)

In [0]:
df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [0]:
df['age'].show()
# We have to select the column we want to see the values

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1241169891005155> in <module> 
 ----> 1 df [ 'age' ] . show ( ) 
 2 # We have to select the column we want to see the values 

 TypeError : 'Column' object is not callable

In [0]:
df.select('age')

Out[21]: DataFrame[age: int]

In [0]:
df.select('age').show()

+----+
 age|
+----+
null|
 30|
 19|
+----+

In [0]:
df.head(2) #df.head returns a list of objects

Out[24]: [Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [0]:
df.head(2)[1]

Out[26]: Row(age=30, name='Andy')

In [0]:
df1 = df.select(['age', 'name'])

In [0]:
df1.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [0]:
# Adding a new columns to the dataframe
df.withColumn('new_age', df['age']*2).show()

+----+-------+-------+
 age| name|new_age|
+----+-------+-------+
null|Michael| null|
 30| Andy| 60|
 19| Justin| 38|
+----+-------+-------+

In [0]:
df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [0]:
# Renaming a column
df.withColumnRenamed('age', "my_age").show()

+------+-------+
my_age| name|
+------+-------+
 null|Michael|
 30| Andy|
 19| Justin|
+------+-------+

In [0]:
df.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [0]:
# Using Spark with SQL

In [0]:
df.createOrReplaceTempView("people")
results = spark.sql("SELECT * FROM people")

In [0]:
results.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [0]:
new_results = spark.sql("SELECT * FROM people WHERE age = 30")

In [0]:
new_results.show()

+---+----+
age|name|
+---+----+
 30|Andy|
+---+----+